# Creating the data to test in STATA and Python

In [1]:
import warnings
warnings.filterwarnings("ignore") # I suppressed the warnings to make it easier to read, we should still try to fix them
import numpy as np
import pandas as pd
from ujive1 import *
from ujive2 import *
from tsls import * 

#Pick a vector length:
n = 1000

#Getting our Z's and making a Z matrix:
Z = np.random.randn(n, 1)
column_of_ones = np.ones((Z.shape[0], 1))
Z = np.hstack((column_of_ones, Z))
#pprint(Z)

#Parameter vectors:
α = np.array([1,1])
β = np.array([1,2])
#pprint(α)
#pprint(β)

#Error terms:
e1 = np.random.normal(0,5,n)
e2 = np.random.normal(0,5,n)
δ = np.random.normal(0,1)
ε = 5*e1 - 5*e2 + δ

#Making our endogenous variable:
x = np.dot(Z,α) + .2*e1
X = np.column_stack((column_of_ones, x))
#pprint(X)

#Outcome vector:
Y = np.dot(X,β) + ε

#OLS benchmark:
bhat_ols = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T, Y))

# Check to see if the Z'Z matrix is invertible
if np.linalg.matrix_rank(Z.T @ Z) == Z.shape[1]:  # Should be True
    print("Z'Z is invertible")
else:
    print("Z'Z is not invertible")

cond_number = np.linalg.cond(Z.T @ Z)
print(f"Condition number of Z.T @ Z: {cond_number}")
if cond_number > 1e10:  # Threshold for ill-conditioning
    raise ValueError("Z.T @ Z is ill-conditioned and may cause numerical instability.")


#2sls comparison:
Zt_Z = np.dot(Z.T, Z)
Zt_Z_inv = np.linalg.inv(Zt_Z)
pz = np.dot(np.dot(Z, Zt_Z_inv), Z.T)
proj_x = np.dot(pz, X)
first = np.linalg.inv(np.dot(proj_x.T, X))
second = np.dot(proj_x.T, Y)
bhat_2sls = np.dot(first, second)


ujive1 = UJIVE1(Y,X,Z,talk=False)
ujive2 = UJIVE2(Y,X,Z,talk=False)
#tsls = TSLS(Y,X,Z)

# Combine matrices into a single DataFrame
df = pd.DataFrame({
    "Y": Y,  # Outcome vector
    **{f"X{i}": X[:, i] for i in range(X.shape[1])},  # Endogenous variables
    **{f"Z{i}": Z[:, i] for i in range(Z.shape[1])}   # Instrumental variables
})

# Save the DataFrame to a CSV file
df.to_csv('data.csv', index=False)

#Compare them:
print("OLS:", bhat_ols[1])
print("2SLS:", bhat_2sls[1])
#print("Jive 1:", ujive1['beta'])
print(ujive1.summary())
#print("Jive 2:",ujive2['beta'])
print(ujive2.summary())

Normally this estimator is used when Z has more columns than X. In this case Z has 2 columns and X has 2 columns.


Z'Z is invertible
Condition number of Z.T @ Z: 1.050409824061365
OLS: 13.912847917014005
2SLS: 2.146124102422597

UJIVE1 Regression Results
 Coefficient  Std. Error    t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
   -1.004868    1.550417 -0.648127 0.517052       -4.047318         2.037583
    2.098025    1.097458  1.911713 0.056199       -0.055566         4.251616
--------------------------------------------------------------------------------
R-squared: 0.083486
Adjusted R-squared: 0.082567
F-statistic: 7.413349
Root MSE: 34.549771
None

UJIVE2 Regression Results
 Coefficient  Std. Error    t-stat    P>|t|  Conf. Int. Low  Conf. Int. High
   -1.008310    1.550372 -0.650367 0.515605       -4.050673         2.034052
    2.101385    1.097405  1.914867 0.055795       -0.052102         4.254872
--------------------------------------------------------------------------------
R-squared: 0.083609
Adjusted R-squared: 0.082690
F-statistic: 7.438109
Root MSE: 34.547456
None


In [2]:
(Z @ np.linalg.inv(np.dot(Z.T,Z)))

array([[ 0.00099394, -0.00086954],
       [ 0.00100659,  0.00094523],
       [ 0.00100996,  0.00142938],
       ...,
       [ 0.00100114,  0.00016376],
       [ 0.00099215, -0.00112617],
       [ 0.00098985, -0.00145629]], shape=(1000, 2))

In [3]:
(Z @ np.linalg.pinv(np.dot(Z.T,Z))) @ Z.T

array([[ 0.00179265,  0.00013835, -0.00030298, ...,  0.00085072,
         0.00202659,  0.00232752],
       [ 0.00013835,  0.00193665,  0.0024164 , ...,  0.00116228,
        -0.00011595, -0.00044308],
       [-0.00030298,  0.0024164 ,  0.00314189, ...,  0.0012454 ,
        -0.00068754, -0.00118222],
       ...,
       [ 0.00085072,  0.00116228,  0.0012454 , ...,  0.00102812,
         0.00080666,  0.00074998],
       [ 0.00202659, -0.00011595, -0.00068754, ...,  0.00080666,
         0.00232957,  0.00271932],
       [ 0.00232752, -0.00044308, -0.00118222, ...,  0.00074998,
         0.00271932,  0.00322332]], shape=(1000, 1000))